## !! Placer ce notebook à la racine pour importer les modules !!

In [1]:
from pymongo import MongoClient

client = MongoClient("localhost", 27017)
dbase  = client["test_presse"]
collec = dbase["test"]

In [8]:
import pprint

req = collec.find({"auteur": "BFMTV.COM"})
for g in req:
    pprint.pprint(g)
    print("=====================================")

{'_id': ObjectId('5eb41ce913782800507fd3cc'),
 'auteur': 'BFMTV.COM',
 'date': '2020-05-05',
 'extrait': 'Dès le début du confinement, beaucoup de municipalités avaient '
            'rendu gratuit le stationnement dans les rues. Mais cette mesure '
            'prendra en grande majorité fin ...',
 'heure': '14:00:00',
 'lien': 'https://news.google.com/articles/CBMifGh0dHBzOi8vYXV0by5iZm10di5jb20vYWN0dWFsaXRlL2EtcGFydGlyLWR1LTExLW1haS1sZS1zdGF0aW9ubmVtZW50LXJlZGV2aWVudC1wYXlhbnQtZGFucy1kZS1ub21icmV1c2VzLXZpbGxlcy0xOTA4MjkwLmh0bWzSAQA?hl=fr&gl=FR&ceid=FR%3Afr',
 'matrice': {'11': 1,
             'beaucoup': 1,
             'confinement': 1,
             'debut': 1,
             'fin': 1,
             'grande': 1,
             'gratuit': 1,
             'mai': 1,
             'majorite': 1,
             'mesure': 1,
             'municipalites': 1,
             'nombreuses': 1,
             'partir': 1,
             'payant': 1,
             'prendra': 1,
             'redevient': 1,
  

In [9]:
# Liste des auteurs les plus fréquents

pipeline = [
    {"$group": {"_id": "$auteur", 
                "total": {"$sum": 1}}},
    {"$sort": {"total": -1}}
    ]  

agg = collec.aggregate(pipeline)

for x in agg:
    print(x)

{'_id': '20 Minutes', 'total': 82}
{'_id': 'Sud Ouest', 'total': 75}
{'_id': 'Le Figaro', 'total': 66}
{'_id': 'Ouest-France', 'total': 64}
{'_id': 'Le Parisien', 'total': 60}
{'_id': 'Le Monde', 'total': 57}
{'_id': 'Actu Orange', 'total': 42}
{'_id': 'LCI', 'total': 39}
{'_id': 'RTL.fr', 'total': 37}
{'_id': 'franceinfo', 'total': 36}
{'_id': 'BFMTV.COM', 'total': 35}
{'_id': 'Boursorama', 'total': 33}
{'_id': 'France Bleu', 'total': 32}
{'_id': 'Yahoo Actualités', 'total': 31}
{'_id': "L'Équipe.fr", 'total': 29}
{'_id': 'Le HuffPost', 'total': 29}
{'_id': 'Franceinfo', 'total': 28}
{'_id': 'BFMTV', 'total': 26}
{'_id': 'FRANCE 24', 'total': 26}
{'_id': 'LaDepeche.fr', 'total': 24}
{'_id': 'Foot Mercato', 'total': 24}
{'_id': 'Le Point', 'total': 22}
{'_id': 'Libération', 'total': 21}
{'_id': 'actu.fr', 'total': 20}
{'_id': 'Maxifoot', 'total': 19}
{'_id': 'Voici', 'total': 18}
{'_id': 'La Provence', 'total': 18}
{'_id': 'CNEWS.fr', 'total': 17}
{'_id': 'Les Échos', 'total': 17}
{'_i

In [ ]:
# Récupération des données manquantes
import time

import requests
from bs4 import BeautifulSoup as bs

from pack.traitement import Traitement

debut = time.time()

sorties = collec.find({"auteur": "20 Minutes"})

total = 0
cpt = 0
for sortie in sorties:
    
    total += 1
    
    if sortie["texte"] == "--":
        cpt += 1
        page_response = requests.get(sortie["lien"])
        soupe = bs(page_response.content, "html.parser")
        
        blocs_2 = soupe.find("div", {"class": "lt-endor-body"})
        if blocs_2 == None:
            break
        paragraphes = blocs_2.find_all("p")
        texte = ""
        for para in paragraphes:
            texte += para.text + "\n"
        sortie["texte"] = texte
            
        # Instanciation de la classe de traitement
        trait = Traitement()
        
        # Concaténation
        #texte = trait.concatenation(sortie)
        
        chaine = ""
        chaine += sortie["titre"] + " "
        if sortie["extrait"][:20] == sortie["texte"][:20]:
            chaine += sortie["texte"]
        else:
            chaine += sortie["extrait"] + " "
            chaine += sortie["texte"]
        
            
        # Calucl de la positivé
        positivite = trait.positivite(texte)
        
        # Calucl de la subjecticité
        subjectivite = trait.subjectivite(texte)
        
        # Conversion en liste de phrases (format attendu pour la création de la matrice de termes)
        parts = chaine.split(". ")
        
        # Nettoyage du texte
        liste_parts = []
        for part in parts:

            # Lemmatisation
            # TODO améliorer le temps de traitement
            part = trait.lemmatisation(part)

            # Nettoyage
            part = trait.nettoyage(part)
            
            liste_parts.append(part)
        
        # Calucl de la matrice de termes
        mat = trait.matrice(liste_parts)
        mat = mat.to_dict()
         
        # Enregistrement dans la base de données
        filtre = {"titre": sortie["titre"],
                  "auteur": sortie["auteur"],
                  "date": sortie["date"]}
        
        updata = {"texte": texte,
                  "positivite": positivite,
                  "subjectivite": subjectivite,
                  "matrice": mat}
        
        collec.update_one(filtre, {"$set": updata})
        
        temps = time.time() - debut
        temps = time.strftime('%Hh, %Mm %Ss', time.gmtime(temps))
        print("Nb d'enregistrement :", cpt, "/", total, "articles, en :", temps)
        import pprint
        res = collec.find(filtre)
        pprint.pprint(list(res)[0])
        print("==========================================")

In [ ]:
# Enregistrement dans la base de données
print("Fin")

In [ ]:
from pack.traitement import Traitement


In [2]:
# Suppression des doublons

res = collec.find()

for x in res:
    data = {"titre": x["titre"],
            "auteur": x["auteur"]}
    q = collec.find(data)
    print(x)
    print("============================================")

{'_id': ObjectId('5eb41ce813782800507fd3a5'), 'titre': 'EN DIRECT - Déconfinement : suivez les annonces de Philippe', 'auteur': 'RTL.fr', 'date': '2020-05-07', 'heure': '12:30:00', 'extrait': 'MINUTE PAR MINUTE - Le Premier ministre fixe les toutes dernières conditions pour la première étape de déconfinement le 11 mai prochain.', 'lien': 'https://news.google.com/articles/CBMicWh0dHBzOi8vd3d3LnJ0bC5mci9hY3R1L3BvbGl0aXF1ZS9kaXJlY3QtdmlkZW8tZGVjb25maW5lbWVudC1lZG91YXJkLXBoaWxpcHBlLWNvcm9uYXZpcnVzLTExLW1haS1jYXJ0ZS03ODAwNDg5NTY00gF1aHR0cHM6Ly93d3cucnRsLmZyL2FjdHUvcG9saXRpcXVlL2RpcmVjdC12aWRlby1kZWNvbmZpbmVtZW50LWVkb3VhcmQtcGhpbGlwcGUtY29yb25hdmlydXMtMTEtbWFpLWNhcnRlLTc4MDA0ODk1NjQvYW1w?hl=fr&gl=FR&ceid=FR%3Afr', 'texte': '--', 'matrice': {'deconfinement': 2, 'minute': 2, 'toutes': 1, 'fixe': 1, 'annonces': 1, 'conditions': 1, 'dernieres': 1, 'direct': 1, 'etape': 1, 'mai': 1, 'suivez': 1, 'ministre': 1, 'philippe': 1, 'premier': 1, 'premiere': 1, 'prochain': 1, '11': 1}}
{'_id': ObjectId('